In [2]:
!pip install tqdm



Defaulting to user installation because normal site-packages is not writeable


In [3]:
import cv2
import os
import numpy as np
from tqdm import tqdm

# Define dataset paths
DATASET_PATH = r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\train"  # Update this with your actual dataset path
PROCESSED_PATH = r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\processed_train"

# Function to preprocess images
def preprocess_images(input_folder, output_folder, img_size=(64, 64)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for img_name in tqdm(os.listdir(input_folder), desc="Processing images"):
        img_path = os.path.join(input_folder, img_name)
        image = cv2.imread(img_path)

        if image is None:
            continue  # Skip unreadable images

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Resize to a fixed size
        resized = cv2.resize(gray, img_size)

        # Save preprocessed image
        cv2.imwrite(os.path.join(output_folder, img_name), resized)

# Run preprocessing
preprocess_images(DATASET_PATH, PROCESSED_PATH)
print("Preprocessing completed.")

Processing images: 100%|███████████████████████████████████████████████████████████| 5082/5082 [01:17<00:00, 65.87it/s]

Preprocessing completed.


In [4]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from sklearn.cluster import KMeans

# Define paths
input_folder = r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\processed_train"
output_folder_drowsy = r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\both\drowsy"
output_folder_non_drowsy = r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\both\non-drowsy"

# Create output directories if they don't exist
os.makedirs(output_folder_drowsy, exist_ok=True)
os.makedirs(output_folder_non_drowsy, exist_ok=True)

# Load pre-trained MobileNetV2 model (without the classification head)
base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

# Function to extract features
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    features = base_model.predict(img_array)
    return features.flatten()

# Extract features from all images
image_paths = [os.path.join(input_folder, img) for img in os.listdir(input_folder) if img.endswith((".jpg", ".png"))]
features = np.array([extract_features(img) for img in image_paths])

# Apply K-Means clustering
kmeans = KMeans(n_clusters=2, random_state=42)
labels = kmeans.fit_predict(features)

# Move images to respective folders
for img_path, label in zip(image_paths, labels):
    if label == 0:
        shutil.move(img_path, os.path.join(output_folder_drowsy, os.path.basename(img_path)))
    else:
        shutil.move(img_path, os.path.join(output_folder_non_drowsy, os.path.basename(img_path)))

print("Classification complete. Images are sorted into 'drowsy' and 'non-drowsy' folders.")


C:\Users\HP\AppData\Local\Temp\ipykernel_4800\2331101270.py:19: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━

In [5]:
import os

# Define paths
output_folder_drowsy =r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\both\drowsy"
output_folder_non_drowsy = r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\both\non-drowsy"

# Count the number of images in each folder
num_drowsy = len([img for img in os.listdir(output_folder_drowsy) if img.endswith((".jpg", ".png"))])
num_non_drowsy = len([img for img in os.listdir(output_folder_non_drowsy) if img.endswith((".jpg", ".png"))])

# Print the counts
print(f"Number of Drowsy Images: {num_drowsy}")
print(f"Number of Non-Drowsy Images: {num_non_drowsy}")

# Check balance
if abs(num_drowsy - num_non_drowsy) / max(num_drowsy, num_non_drowsy) > 0.2:  # More than 20% imbalance
    print("⚠️ The dataset is imbalanced!")
else:
    print("✅ The dataset is balanced.")


Number of Drowsy Images: 1696
Number of Non-Drowsy Images: 845
⚠️ The dataset is imbalanced!


In [6]:
import random
import shutil

# Paths
drowsy_images = [os.path.join(output_folder_drowsy, img) for img in os.listdir(output_folder_drowsy)]
non_drowsy_images = [os.path.join(output_folder_non_drowsy, img) for img in os.listdir(output_folder_non_drowsy)]

# Balance by undersampling the majority class
num_to_keep = len(non_drowsy_images)  # Match the minority class
drowsy_images_to_remove = random.sample(drowsy_images, len(drowsy_images) - num_to_keep)

# Move extra drowsy images to a separate folder (optional: instead of deleting)
backup_folder = r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\both\backup_drowsy"
os.makedirs(backup_folder, exist_ok=True)

for img in drowsy_images_to_remove:
    shutil.move(img, os.path.join(backup_folder, os.path.basename(img)))

print(f"Balanced dataset: {num_to_keep} drowsy and {len(non_drowsy_images)} non-drowsy images.")


Balanced dataset: 845 drowsy and 845 non-drowsy images.


In [7]:
import os

# Define paths
output_folder_drowsy =r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\both\drowsy"
output_folder_non_drowsy = r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\both\non-drowsy"

# Count the number of images in each folder
num_drowsy = len([img for img in os.listdir(output_folder_drowsy) if img.endswith((".jpg", ".png"))])
num_non_drowsy = len([img for img in os.listdir(output_folder_non_drowsy) if img.endswith((".jpg", ".png"))])

# Print the counts
print(f"Number of Drowsy Images: {num_drowsy}")
print(f"Number of Non-Drowsy Images: {num_non_drowsy}")

# Check balance
if abs(num_drowsy - num_non_drowsy) / max(num_drowsy, num_non_drowsy) > 0.2:  # More than 20% imbalance
    print("⚠️ The dataset is imbalanced!")
else:
    print("✅ The dataset is balanced.")


Number of Drowsy Images: 845
Number of Non-Drowsy Images: 845
✅ The dataset is balanced.


In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import pickle

# Paths
input_folder = r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\both"
svm_model_path = "svm_model_with_rgb.pkl"

# Load pre-trained MobileNetV2 for feature extraction
base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

# Function to extract features from an image
def extract_features(img_path):
    # Load image for MobileNetV2 feature extraction
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    features = base_model.predict(img_array)
    
    # Flatten MobileNetV2 features
    mobilenet_features = features.flatten()
    
    # Extract RGB features: mean of each channel
    img = np.array(img)  # Load the image again for RGB analysis
    r_mean = np.mean(img[:, :, 0])  # Mean of the red channel
    g_mean = np.mean(img[:, :, 1])  # Mean of the green channel
    b_mean = np.mean(img[:, :, 2])  # Mean of the blue channel
    
    # Combine MobileNetV2 features with RGB features
    combined_features = np.concatenate([mobilenet_features, [r_mean, g_mean, b_mean]])
    
    return combined_features

# Load dataset and extract features
image_paths = []
labels = []

# Assuming images are stored in "drowsy" and "non-drowsy" subfolders
for category in ["drowsy", "non-drowsy"]:
    category_path = os.path.join(input_folder, category)
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        image_paths.append(img_path)
        labels.append(0 if category == "drowsy" else 1)  # 0 = Drowsy, 1 = Non-Drowsy

# Extract features from all images
features = np.array([extract_features(img) for img in image_paths])
labels = np.array(labels)

# Split dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train an SVM classifier
svm = SVC(kernel="linear", probability=True)
svm.fit(X_train, y_train)

# Save the trained model
with open(svm_model_path, "wb") as file:
    pickle.dump(svm, file)

print("SVM model trained and saved successfully!")

# Evaluate the model
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Print detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Drowsy", "Non-Drowsy"]))

# Function to predict drowsiness for a new image
def predict_drowsiness(img_path):
    features = extract_features(img_path).reshape(1, -1)
    label = svm.predict(features)[0]
    return "Drowsy" if label == 0 else "Non-Drowsy"

# Example usage
image_path = r"C:\Users\HP\Downloads\Driver Monitoring System.v1-v1_4mar2024.voc\test\1-2053-_jpg.rf.2af4bf2957193f121df719e6ec7ab14b.jpg"
prediction = predict_drowsiness(image_path)
print(f"The image is classified as: {prediction}")


C:\Users\HP\AppData\Local\Temp\ipykernel_4800\3327611173.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [2]:
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image

# Load the trained SVM model
with open("svm_model_with_rgb.pkl", "rb") as file:
    svm = pickle.load(file)

# Load MobileNetV2 model (same as used during training)
base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

# Function to extract features from an image
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    
    # Extract MobileNetV2 features
    features = base_model.predict(img_array).flatten()

    # Extract RGB mean values
    img = np.array(img)
    r_mean = np.mean(img[:, :, 0])  # Red channel
    g_mean = np.mean(img[:, :, 1])  # Green channel
    b_mean = np.mean(img[:, :, 2])  # Blue channel

    # Combine MobileNetV2 features with RGB means
    combined_features = np.concatenate([features, [r_mean, g_mean, b_mean]])

    return combined_features

# Function to predict drowsiness for a new image
def predict_drowsiness(img_path):
    features = extract_features(img_path).reshape(1, -1)
    label = svm.predict(features)[0]
    return "Drowsy" if label == 0 else "Non-Drowsy"

# Provide the path to the image you want to test
test_image_path = r"C:\Users\HP\Pictures\Camera Roll\WIN_20250312_13_57_17_Pro.jpg"
prediction = predict_drowsiness(test_image_path)

print(f"The image is classified as: {prediction}")


C:\Users\HP\AppData\Local\Temp\ipykernel_20736\2350284584.py:12: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
The image is classified as: Non-Drowsy
